# 二阶魔方数学

In [1]:
import numpy as np

# 基本几何
## 面，主轴
## 顶点，块

In [2]:
# 尺度 -1， 1
scale = [-1, 1] 

# 面、转轴向量
faces = []
for code_java in scale:
    faces.append([code_java, 0, 0])
    faces.append([0, code_java, 0])
    faces.append([0, 0, code_java])
print(faces)

# 顶点、块坐标
blocks = []

for x in scale:
    for y in scale:
        for z in scale:
            blocks.append([x, y, z])
print(blocks)

[[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[-1, -1, -1], [-1, -1, 1], [-1, 1, -1], [-1, 1, 1], [1, -1, -1], [1, -1, 1], [1, 1, -1], [1, 1, 1]]


# 旋转矩阵

In [3]:
##
##  Wiki 上绕轴旋转的公式
##  URL_ADDRESS##  https://en.wikipedia.org/wiki/Rotation_matrix#Rotation_matrix_from_axis_and_angle


def getRotateMatrix(v, theta ):
    n = np.sqrt(np.dot(v,v))
    [x,y,z] = v/n
    s = np.sin(theta)
    c = np.cos(theta)
    c1 = 1 - c
    return np.array([[c + c1 * x * x, c1 * x * y - s * z, c1 * x * z + s * y],
                    [c1 * y * x + s * z, c + c1 * y * y, c1 * y * z - s * x],
                    [c1 * z * x - s * y, c1 * z * y + s * x, c + c1 * z * z]])


# 绕轴旋转的矩阵，用于面旋转关系

In [4]:
face_rotates = []
for face in faces:
    mat = getRotateMatrix(face, np.pi * 2 / 4)
    face_rotates.append(np.array(mat,np.int16))
print(face_rotates)

[array([[ 1,  0,  0],
       [ 0,  0,  1],
       [ 0, -1,  0]], dtype=int16), array([[ 0,  0, -1],
       [ 0,  1,  0],
       [ 1,  0,  0]], dtype=int16), array([[ 0,  1,  0],
       [-1,  0,  0],
       [ 0,  0,  1]], dtype=int16), array([[ 1,  0,  0],
       [ 0,  0, -1],
       [ 0,  1,  0]], dtype=int16), array([[ 0,  0,  1],
       [ 0,  1,  0],
       [-1,  0,  0]], dtype=int16), array([[ 0, -1,  0],
       [ 1,  0,  0],
       [ 0,  0,  1]], dtype=int16)]


# 绕顶点旋转的矩阵，用于块朝向

In [5]:


corner_rotates = []
for corner in blocks:
    mat = getRotateMatrix(corner, np.pi * 2 / 3)
    corner_rotates.append(np.array(mat,np.int16))
print(corner_rotates)


[array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]], dtype=int16), array([[ 0,  0, -1],
       [ 1,  0,  0],
       [ 0, -1,  0]], dtype=int16), array([[ 0,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0]], dtype=int16), array([[ 0, -1,  0],
       [ 0,  0,  1],
       [-1,  0,  0]], dtype=int16), array([[ 0,  0, -1],
       [-1,  0,  0],
       [ 0,  1,  0]], dtype=int16), array([[ 0, -1,  0],
       [ 0,  0, -1],
       [ 1,  0,  0]], dtype=int16), array([[ 0,  1,  0],
       [ 0,  0, -1],
       [-1,  0,  0]], dtype=int16), array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]], dtype=int16)]


### 校验绕顶点旋转矩阵

In [6]:
# check corner_rotates
for i in range(8):
    r1 = np.matmul(corner_rotates[i],np.array([blocks[i][0],0,0]).reshape((3,1)))
    r2 = np.matmul(corner_rotates[i],r1)
    r3 = np.matmul(corner_rotates[i],r2)
    print(r1.reshape(1,3))
    print(r2.reshape(1,3))
    print(r3.reshape(1,3))
    print(blocks[i])

[[ 0  0 -1]]
[[ 0 -1  0]]
[[-1  0  0]]
[-1, -1, -1]
[[ 0 -1  0]]
[[0 0 1]]
[[-1  0  0]]
[-1, -1, 1]
[[0 1 0]]
[[ 0  0 -1]]
[[-1  0  0]]
[-1, 1, -1]
[[0 0 1]]
[[0 1 0]]
[[-1  0  0]]
[-1, 1, 1]
[[ 0 -1  0]]
[[ 0  0 -1]]
[[1 0 0]]
[1, -1, -1]
[[0 0 1]]
[[ 0 -1  0]]
[[1 0 0]]
[1, -1, 1]
[[ 0  0 -1]]
[[0 1 0]]
[[1 0 0]]
[1, 1, -1]
[[0 1 0]]
[[0 0 1]]
[[1 0 0]]
[1, 1, 1]


# 面与块的关系
## 各个角块朝外的3个面
## 定义标准顺序 可用于确定块的朝向(会影响到映射数组数值)

In [7]:
block_associated_faces = []
for i in range(8):
    associated_face = []
    if(i < 4):
        associated_face.append(0)
        v1 = np.matmul(corner_rotates[i], np.array(faces[0]).reshape((3,1)))
    
    else: 
        associated_face.append(3)

        v1 = np.matmul(corner_rotates[i], np.array(faces[3]).reshape((3,1)))
    v2 = np.matmul(corner_rotates[i], v1)

    for j in range(6):
        if (np.dot(v1.reshape((1,3)), faces[j]) == 1):
            associated_face.append(j)

    for j in range(6):
        if (np.dot(v2.reshape((1,3)), faces[j]) == 1):
            associated_face.append(j)
    

    block_associated_faces.append(associated_face)

print(block_associated_faces)

[[0, 2, 1], [0, 1, 5], [0, 4, 2], [0, 5, 4], [3, 1, 2], [3, 5, 1], [3, 2, 4], [3, 4, 5]]


### 校验

In [8]:
# check blcok associated faces
for i in range(8):
    vsum = np.zeros(3)
    for j in range(3):
        vsum += faces[block_associated_faces[i][j]]
    print(vsum)
    print(blocks[i])

[-1. -1. -1.]
[-1, -1, -1]
[-1. -1.  1.]
[-1, -1, 1]
[-1.  1. -1.]
[-1, 1, -1]
[-1.  1.  1.]
[-1, 1, 1]
[ 1. -1. -1.]
[1, -1, -1]
[ 1. -1.  1.]
[1, -1, 1]
[ 1.  1. -1.]
[1, 1, -1]
[1. 1. 1.]
[1, 1, 1]


## 生成旋转映射数组
### TODO 解释生成逻辑

In [9]:
rotate_mappings = []
for i in range(6):
    maplet = []
    for j in range(8):
        if(np.dot(faces[i], blocks[j]) == 1):
            maplet.append(j)
            break
    
    v1 = np.matmul(face_rotates[i], np.array(blocks[maplet[0]]).reshape((3,1)))
    for j in range(8):
        if(np.dot(v1.reshape(1,3), blocks[j]) == 3):
            maplet.append(j)
    
    v2 = np.matmul(face_rotates[i], v1)
    for j in range(8):
        if(np.dot(v2.reshape(1,3), blocks[j]) == 3):
            maplet.append(j)
    
    v3 = np.matmul(face_rotates[i], v2)
    for j in range(8):
        if(np.dot(v3.reshape(1,3), blocks[j]) == 3):
            maplet.append(j)
    
    # print(maplet)
    d0s = np.array(list(map(lambda idx:faces[idx] ,block_associated_faces[maplet[0]]))).transpose()
    print(d0s)
    print(np.matmul(face_rotates[i], d0s))
    d1s = np.array(list(map(lambda idx:faces[idx] ,block_associated_faces[maplet[1]]))).transpose()
    print(d1s) 
    print(np.matmul(face_rotates[i], d1s))
    d2s = np.array(list(map(lambda idx:faces[idx] ,block_associated_faces[maplet[2]]))).transpose()
    print(d2s) 
    print(np.matmul(face_rotates[i], d2s))
    d3s = np.array(list(map(lambda idx:faces[idx] ,block_associated_faces[maplet[3]]))).transpose()
    print(d3s) 
    print(np.matmul(face_rotates[i], d2s))
    
    d00 = np.array(faces[block_associated_faces[maplet[0]][0]]).reshape(3,1)
    d0 = np.matmul(face_rotates[i], d00)

    for j in range(3):
        if(np.dot(d0.reshape((1,3)), faces[block_associated_faces[maplet[1]][j]]) == 1):
            maplet.append(j)

    d10 = np.array(faces[block_associated_faces[maplet[1]][0]]).reshape(3,1)
    d1 = np.matmul(face_rotates[i], d10)
    for j in range(3):
        if(np.dot(d1.reshape((1,3)), faces[block_associated_faces[maplet[2]][j]]) == 1):
            maplet.append(j)

    d20 = np.array(faces[block_associated_faces[maplet[2]][0]]).reshape(3,1)
    d2 = np.matmul(face_rotates[i], d20)
    for j in range(3):
        if(np.dot(d2.reshape((1,3)), faces[block_associated_faces[maplet[3]][j]]) == 1):
            maplet.append(j)

    d30 = np.array(faces[block_associated_faces[maplet[3]][0]]).reshape(3,1)
    d3 = np.matmul(face_rotates[i], d30)
    for j in range(3):
        if(np.dot(d3.reshape((1,3)), faces[block_associated_faces[maplet[0]][j]]) == 1):
            maplet.append(j)
    
     
    print(maplet)
    rotate_mappings.append(maplet)
print(rotate_mappings)

[[-1  0  0]
 [ 0  0 -1]
 [ 0 -1  0]]
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
[[-1  0  0]
 [ 0  0  1]
 [ 0  1  0]]
[[-1  0  0]
 [ 0  0  1]
 [ 0  1  0]]
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
[0, 1, 3, 2, 0, 0, 0, 0]
[[-1  0  0]
 [ 0  0 -1]
 [ 0 -1  0]]
[[ 0  1  0]
 [ 0  0 -1]
 [-1  0  0]]
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
[[ 0  0  1]
 [ 0 -1  0]
 [ 1  0  0]]
[[ 1  0  0]
 [ 0  0 -1]
 [ 0  1  0]]
[[ 0 -1  0]
 [ 0  0 -1]
 [ 1  0  0]]
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
[[ 0 -1  0]
 [ 0  0 -1]
 [ 1  0  0]]
[0, 4, 5, 1, 2, 1, 2, 1]
[[-1  0  0]
 [ 0  0 -1]
 [ 0 -1  0]]
[[ 0  0 -1]
 [ 1  0  0]
 [ 0 -1  0]]
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
[[ 0  1  0]
 [ 1  0  0]
 [ 0  0 -1]]
[[ 1  0  0]
 [ 0  0  1]
 [ 0 -1  0]]
[[ 0  0  1]
 [-1  0  0]
 [ 0 -1  0]]
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
[[ 0  0  1]
 [-1  0  0]
 [ 0 -1  0]]
[0, 2, 6, 4, 1, 2, 1, 2]
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]


### 映射数组的生成，np格式的保存，读取

In [10]:
mmm = list(map( lambda line : 
        [
            
            [line[0], line[1],line[4]],
            [line[1], line[2],line[5]],
            [line[2], line[3],line[6]],
            [line[3], line[0],line[7]],

        ]
        ,rotate_mappings))
np.save("mapping2025.npy", mmm)

In [11]:
mmm2 = np.load("mapping2025.npy")
print(mmm2)

[[[0 1 0]
  [1 3 0]
  [3 2 0]
  [2 0 0]]

 [[0 4 2]
  [4 5 1]
  [5 1 2]
  [1 0 1]]

 [[0 2 1]
  [2 6 2]
  [6 4 1]
  [4 0 2]]

 [[4 6 0]
  [6 7 0]
  [7 5 0]
  [5 4 0]]

 [[2 3 1]
  [3 7 2]
  [7 6 1]
  [6 2 2]]

 [[1 5 2]
  [5 7 1]
  [7 3 2]
  [3 1 1]]]


## 生成旋转数组的声明代码（Java）

In [12]:
blockjoin = lambda block : f"""new int[]{{ { ','.join(map (lambda item: f"{item}",block))} }}"""
# print(blockjoin([1,2,3]))

newline = '\n'
code_java = f"""new int[][][]{{{ f',{newline}'.join(map(lambda line :
                   f'''new int[][]{{{','.join(map(blockjoin, line ))}}}''',mmm2))
     }}}"""

print(f"int[][][] mapping = {code_java};")

int[][][] mapping = new int[][][]{new int[][]{new int[]{ 0,1,0 },new int[]{ 1,3,0 },new int[]{ 3,2,0 },new int[]{ 2,0,0 }},
new int[][]{new int[]{ 0,4,2 },new int[]{ 4,5,1 },new int[]{ 5,1,2 },new int[]{ 1,0,1 }},
new int[][]{new int[]{ 0,2,1 },new int[]{ 2,6,2 },new int[]{ 6,4,1 },new int[]{ 4,0,2 }},
new int[][]{new int[]{ 4,6,0 },new int[]{ 6,7,0 },new int[]{ 7,5,0 },new int[]{ 5,4,0 }},
new int[][]{new int[]{ 2,3,1 },new int[]{ 3,7,2 },new int[]{ 7,6,1 },new int[]{ 6,2,2 }},
new int[][]{new int[]{ 1,5,2 },new int[]{ 5,7,1 },new int[]{ 7,3,2 },new int[]{ 3,1,1 }}};


## 生成旋转数组的声明代码（C）

In [14]:
blockjoin = lambda block : f"""{{ { ','.join(map (lambda item: f"{item}",block))} }}"""

newline = '\n'
code_c = f"""{{{ f',{newline}'.join(map(lambda line :
                   f'''{{{','.join(map(blockjoin, line ))}}}''',mmm2))
     }}}"""

print(f"const int mapping[6][4][3] = {code_c};")

const int mapping[6][4][3] = {{{ 0,1,0 },{ 1,3,0 },{ 3,2,0 },{ 2,0,0 }},
{{ 0,4,2 },{ 4,5,1 },{ 5,1,2 },{ 1,0,1 }},
{{ 0,2,1 },{ 2,6,2 },{ 6,4,1 },{ 4,0,2 }},
{{ 4,6,0 },{ 6,7,0 },{ 7,5,0 },{ 5,4,0 }},
{{ 2,3,1 },{ 3,7,2 },{ 7,6,1 },{ 6,2,2 }},
{{ 1,5,2 },{ 5,7,1 },{ 7,3,2 },{ 3,1,1 }}};
